## Factors against Parallelism

* Startup costs associated with initiating processes
  * May often overwhelm actual processing time (rendering ||ism useless)
  * Involve thread/process creation, data movement
* Interference: slowdown resulting from multiple processors accessing shared resources
  * Resources: memory, I/O, system bus, sub-processors
  * Software synchronization: locks, latches, mutex, barriers
  * Hardware synchronization: cache faults, interrupts
* Skew: when breaking a single task into many smaller tasks, not all tasks may be the same size
  * Not all tasks finish at the same time
  
### Understanding Factors by Analogy



#### Startup

* Szkieletor in Krakow Poland
  * Too expensive to complete or demolish
* Why is this like a parallel computer?
    * It is a parallel living environment
    * The parallel living throughput is 0 because of startup
    
<img src="./images/sz.png" width="300" title="http://en.wikipedia.org/wiki/File:Szkieletor2.jpg" />

#### Interference: 

* Congested intersections
  * mulitple vehicles compete for same resource (lanes in roundabout)
  * others await resources
* This is a parallel driving environment
  * unused capacity (16 outbound lanes) because resource competition (the roundabout) prevents its use
  * this system exhibits a _throughput collapse_ in which more usage reduces flow
  
<img src="./images/traffic.png" width="512" title="http://crowdcentric.net/2011/05/can-you-help-us-solve-the-worlds-traffic-congestion/" />


#### Skew: 

* Completion of parts for assembly
  * throughput: output planes stalled awaiting other parts
* The parallelism is implicit in the parallel construction of all parts
  * the entire system is stalled (seen) awaiting a nose section.

<img src="./images/plane.png" width="512" title="http://www.ainonline.com/?q=aviation-news/dubai-air-show/2011-11-12/" />


In [ ]:
### In Computer: Real things that Degrade Parllelism

* I/O (memory and storage)
  * may be startup (load data before computation)
  * may be interference (awaiting data transfer between parallel tasks)
  * may be skew (await I/O completion of one task)

* Network communication
    * similar to I/O but always involves communication

* Failures—particularly slow/failed processes (often skew)

The HPC community focuses on communication (among processes) as the major source of slowdown.  This is a traditional (I/O and networking) view.

### Communication

* Parallel computation proceeds in phases
  * Compute (evaluate data that you have locally)
* Communicate (exchange data among compute tasks).  Performance is governed by:
  * Latency: fixed cost to send a message
  * Round trip time (speed of light and switching costs)
* Bandwidth: marginal cost to send a message
  * Link capacity
* Latency dominates small messages and bandwidth dominates large
Almost always better to increase message size for performance, but difficult to achieve in practice.

In [ ]:
### Overlapping Communication and I/O

A simplified model 

In [1]:
# synchronous I/O and then compute
def factorial(number):  
    f = 1
    for i in range(2, number+1):
        f *= i
    return f

def io_from_devnull(number):
    with open("/dev/null", "rb") as fh:
        for i in range(number):
            fh.read(1)
    return number

%timeit -n 20 factorial(10000)
%timeit -n 20 io_from_devnull(30000)
%timeit -n 20 factorial(10000); io_from_devnull(30000)

18.5 ms ± 896 µs per loop (mean ± std. dev. of 7 runs, 20 loops each)
20.7 ms ± 554 µs per loop (mean ± std. dev. of 7 runs, 20 loops each)
39.1 ms ± 877 µs per loop (mean ± std. dev. of 7 runs, 20 loops each)


In [4]:
from multiprocessing import Process

p1 = Process(target=factorial, args=(10000,))
p2 = Process(target=io_from_devnull, args=(30000,))
%timeit -n 10 p1.start(); p2.start(); p1.join(); p2.join()


AssertionError: cannot start a process twice

In [5]:
import nest_asyncio
nest_asyncio.apply()

import asyncio  

async def factorial(number):  
    f = 1
    for i in range(2, number+1):
        f *= i
    return f

async def io_from_devnull(number):
    with open("/dev/null", "rb") as fh:
        for i in range(number):
            fh.read(1)
    return number

tasks = [  
    asyncio.ensure_future(factorial(10000)),
    asyncio.ensure_future(io_from_devnull(30000))
]
loop = asyncio.get_event_loop()
%time loop.run_until_complete(asyncio.wait(tasks))  


CPU times: user 60 ms, sys: 0 ns, total: 60 ms
Wall time: 56.4 ms


({<Task finished coro=<factorial() done, defined at <ipython-input-5-8ea35d274505>:6> result=284625968091705451...0000000000000000000>,
  <Task finished coro=<io_from_devnull() done, defined at <ipython-input-5-8ea35d274505>:12> result=30000>},
 set())

In [ ]:
### Bulk Synchronous Processing

A simplified model 